In [20]:
#importing some required libraries
import time
import requests
import os
import tqdm

from newsplease import NewsPlease

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np

import json
import nltk

import pyLDAvis
import pyLDAvis.gensim

import matplotlib.pyplot as plt
%matplotlib inline
pyLDAvis.enable_notebook()

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
def news_lda(date_str,k):
    
    #This is my nytimes developer API key required to make the requests.
    API_KEY="3EhyoUDWeGwW0vTGCY931FOgXOHCGBAj"
    
    date_trimmed=date_trimmed=date_str[6:]+date_str[0:2]+date_str[3:5]
    DATE=date_trimmed
    
    #components of url for api query
    PREFIX="http://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date="
    CENTER="&end_date="
    SUFFIX="&api-key="
    
    #generating the url
    url=PREFIX+date_trimmed+CENTER+date_trimmed+SUFFIX+API_KEY
    print(url)
    
    
    #getting the json file by making request to the servar
    response_json=requests.get(url).json()
    print(response_json.keys())
    
    
    #finding total number of documents
    num_docs=response_json['response']['meta']['hits']
    print(num_docs)

    #adding the documents on first page in a newly created list of all document
    docs=response_json['response']['docs']
    
    
    #since each page contains 10 queries
    if num_docs%10 is 0:
        adjuster=1
    else:
        adjuster=0

    total_pages = int(num_docs/10)+1-adjuster


    # In[33]:


    for page in range(1,total_pages):
        #creating the url, basically page arguement allows us to get different documents
        #page=0 gives first 10, page=1 gives next 10 and so on
        url=PREFIX+DATE+CENTER+DATE+"&page="+str(page)+SUFFIX+API_KEY
        response_json=requests.get(url).json()

        #Okay so nytimes allows only 10 queries in a minute(and only 4000 in a day)
        #so let's wait a minute before making the same query again
        if not 'response' in response_json.keys():
            print("Waiting for 1 minute before making another request")
            time.sleep(60)
            response_json=requests.get(url).json()        

        #adding these documents to our list as well
        new_docs = response_json['response']['docs']
        docs.extend(new_docs)
        
    
    #making the list of web urls    
        web_urls = []

        for doc in docs:
            web_urls.append(doc['web_url'])
            

    #adding articles to the list
    articles=[]

    for i in range(len(docs)):
        try:
            article=NewsPlease.from_url(web_urls[i])
            articles.append(article)
            print("Success in article {}".format(i))
        except:
            print("Encountered HTTPError in one of the pages moving on")
            print(web_urls[i])
    
    #checking if articles files exists from previous run
    filenames = os.listdir()
    if "articles.json" in filenames:
        !rm "articles.json"
    
    articles_in_dic_format = [article.get_dict() for article in articles]
    
    
    #taking the relevant three sections from each article
    simplified_articles = [(str(article['description']) + str(article['text']) + str(article['title']) ) for article in articles_in_dic_format]
    
    #saving this for future se
    with open('articles.json','w') as file:
        json.dump(simplified_articles,file)
        
    with open('articles.json','r') as file:
        data = json.load(file)
        
       
    
    
    #now the lda part
    
    def lemmatize_stemming(text):
        return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

    stemmer = SnowballStemmer("english")

    def preprocess(text):
        result = []
        for token in gensim.utils.simple_preprocess(text):
            if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
                result.append(lemmatize_stemming(token))
        return result
    
    
    #processing each doc and adding it to list of processed_docs
    processed_docs = [preprocess(doc) for doc in data]
    
    
    #creating a dictionary maping each word to id
    #note that this is very small data
    #else we should have used MapReduce
    dictionary = gensim.corpora.Dictionary(processed_docs)
            
            
    #right now dont feel I have use of keep_n parameter as number of docs is very low
    #not keeping words that appear in less than 5 documens
    #not keeping words appearing in more than 50 percent documents
    #maybe I would have set no_above lower if we had a larger dataset
    dictionary.filter_extremes(no_below=5, no_above=0.5)
    
    
    #document representation in bag of words format
    #basically each doc is now just words and their frequency without any importance to position
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
    
    
    #reqiire these for tfidf model
    from gensim import corpora, models

    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]

    from pprint import pprint
        
    #training LDA model based on BOW
    lda_model_bow = gensim.models.LdaMulticore(bow_corpus, num_topics=k, id2word=dictionary,batch=False, passes=10, workers=None, iterations=1000)
    
    #getting visualization tool ready
    vis_lda_bow = pyLDAvis.gensim.prepare(lda_model_bow, bow_corpus, dictionary)
    
    pyLDAvis.save_html(vis_lda_bow, '/newvolume/data_transfer/nytimes/templates/lda2.html')
    pyLDAvis.save_html(vis_lda_bow, '/newvolume/data_transfer/nytimes/templates/lda.html')

    
    return
    
    
    
    
    
    


In [ ]:
from flask import Flask, render_template, url_for, request, jsonify
app = Flask(__name__)

@app.route('/')
def hello_world():
   return render_template("headline_correcter.html")


@app.route('/input',methods = ['GET'])
def input_taker():
    if request.method == 'GET':
        date_str = request.args.get('date')
        num_topics = request.args.get('num_topics')
        print(date_str)
        print(num_topics)
        news_lda(date_str,num_topics)
        
    return render_template("lda.html")
    
    

if __name__ == '__main__':
   app.run(host= '0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
203.110.242.24 - - [24/Mar/2019 12:38:37] "GET / HTTP/1.1" 200 -


03-24-2004
5
http://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date=20040324&end_date=20040324&api-key=3EhyoUDWeGwW0vTGCY931FOgXOHCGBAj
dict_keys(['status', 'copyright', 'response'])
251
Waiting for 1 minute before making another request
Waiting for 1 minute before making another request
Success in article 0
Success in article 1
Success in article 2
Success in article 3
Success in article 4
Success in article 5
Success in article 6
Success in article 7
Success in article 8
Success in article 9
Success in article 10
Success in article 11
Success in article 12
Success in article 13
Success in article 14
Success in article 15
Success in article 16
Success in article 17
Success in article 18
Success in article 19
Success in article 20
Success in article 21
Success in article 22
Success in article 23
Success in article 24
Success in article 25
Success in article 26
Success in article 27
Success in article 28
Success in article 29
Success in article 30
Success in article 31
Succe

203.110.242.24 - - [24/Mar/2019 12:53:03] "GET /input?date=03-24-2004&num_topics=5 HTTP/1.1" 200 -
